In [1]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
import sys
from pathlib import Path
from custom_assets import colors


#for jupyter
import os

pio.renderers.default = (
    "browser"  # modify this to plot on something else besides browser
)


In [81]:
#script_dir = Path(__file__).parent  
#data_path = (script_dir / "../../data").resolve()

# for jupyter
script_dir = os.path.abspath('')

import os 
# for silly jupyter
data_path = os.path.join(script_dir, "../../data/")

# Modify the variables below to plot your desired play
input_data_loc = "train/input_2023_w01.csv"
output_data_loc = "train/output_2023_w01.csv"

game_id = 2023090700
play_id = 194
supplementary_file = "supplementary_data.csv"

#tracking_file = data_path / input_data_file
# for silly jupyter
input_data_file = os.path.join(data_path, input_data_loc)
output_data_file = os.path.join(data_path, output_data_loc)
info_file = os.path.join(data_path, supplementary_file)

print(input_data_file)
print(output_data_file)
print(info_file)

/Users/pick/projects/pred-nfl-bdb-2026/code/plotting/../../data/train/input_2023_w01.csv
/Users/pick/projects/pred-nfl-bdb-2026/code/plotting/../../data/train/output_2023_w01.csv
/Users/pick/projects/pred-nfl-bdb-2026/code/plotting/../../data/supplementary_data.csv


In [82]:
df_input = pd.read_csv(input_data_file)
df_output = pd.read_csv(output_data_file)
df_plays = pd.read_csv(info_file)

/var/folders/xj/g4f3qxj923dg9hvptdpcl3mr0000gn/T/ipykernel_66406/3267752177.py:3: DtypeWarning:

Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.



In [83]:
df_input_merged = df_input.merge(df_plays, on=["game_id", "play_id"])
df_output_merged = df_output.merge(df_plays, on=["game_id", "play_id"])

print(df_input_merged.columns)

drop_list = ['penalty_yards',
       'pre_penalty_yards_gained', 'yards_gained', 'expected_points',
       'expected_points_added', 'pre_snap_home_team_win_probability',
       'pre_snap_visitor_team_win_probability',
       'home_team_win_probability_added', 'visitor_team_win_probility_added', 'play_nullified_by_penalty']

df_input_final = df_input_merged.drop(drop_list, axis=1)
df_output_merged = df_output_merged.drop(drop_list, axis=1)

Index(['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id',
       'play_direction', 'absolute_yardline_number', 'player_name',
       'player_height', 'player_weight', 'player_birth_date',
       'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a',
       'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'season',
       'week', 'game_date', 'game_time_eastern', 'home_team_abbr',
       'visitor_team_abbr', 'play_description', 'quarter', 'game_clock',
       'down', 'yards_to_go', 'possession_team', 'defensive_team',
       'yardline_side', 'yardline_number', 'pre_snap_home_score',
       'pre_snap_visitor_score', 'play_nullified_by_penalty', 'pass_result',
       'pass_length', 'offense_formation', 'receiver_alignment',
       'route_of_targeted_receiver', 'play_action', 'dropback_type',
       'dropback_distance', 'pass_location_type', 'defenders_in_the_box',
       'team_coverage_man_zone', 'team_coverage_type', 'penalty_yards',
       'pre_

In [84]:
# Create a mapping of player roles from pre-pass data
# Each player has one role per play
role_mapping = df_input_final[['game_id', 'play_id', 'nfl_id', 'player_role']].drop_duplicates()
df_output_final = df_output_merged.merge(
    role_mapping,
    on=['game_id', 'play_id', 'nfl_id'],
    how='left'
)

df_output_final['player_to_predict'] = True

In [85]:
df_input_focused = df_input_final[
    (df_input_final["play_id"] == play_id) & (df_input_final["game_id"] == game_id)
]

df_output_focused = df_output_final[
    (df_output_final["play_id"] == play_id) & (df_output_final["game_id"] == game_id)
]


# Concat the two dataframes - merge the two segments of the play into one
pass_frame_id = df_input_focused['frame_id'].max() + 1
df_output_focused['frame_id'] = df_output_focused['frame_id'] + df_input_focused['frame_id'].max()
df_focused = pd.concat([df_input_focused, df_output_focused], ignore_index=True)


df_focused

/var/folders/xj/g4f3qxj923dg9hvptdpcl3mr0000gn/T/ipykernel_66406/3567608740.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,offense_formation,receiver_alignment,route_of_targeted_receiver,play_action,dropback_type,dropback_distance,pass_location_type,defenders_in_the_box,team_coverage_man_zone,team_coverage_type
0,2023090700,194,True,44888,1,left,89.0,Alex Anzalone,6-3,240.0,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
1,2023090700,194,True,44888,2,left,89.0,Alex Anzalone,6-3,240.0,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
2,2023090700,194,True,44888,3,left,89.0,Alex Anzalone,6-3,240.0,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
3,2023090700,194,True,44888,4,left,89.0,Alex Anzalone,6-3,240.0,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
4,2023090700,194,True,44888,5,left,89.0,Alex Anzalone,6-3,240.0,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,2023090700,194,True,41325,37,NaN,NaN,NaN,NaN,NaN,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
457,2023090700,194,True,41325,38,NaN,NaN,NaN,NaN,NaN,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
458,2023090700,194,True,41325,39,NaN,NaN,NaN,NaN,NaN,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN
459,2023090700,194,True,41325,40,NaN,NaN,NaN,NaN,NaN,...,SHOTGUN,2x2,ANGLE,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN


In [86]:
# Get General Play Information
absolute_yd_line = df_focused.absolute_yardline_number.values[0]
play_going_right = (
    df_focused.play_direction.values[0] == "right"
)  # 0 if left, 1 if right

line_of_scrimmage = absolute_yd_line
ball_land_x = df_focused.ball_land_x.values[0]
ball_land_y = df_focused.ball_land_y.values[0]


print(f'Play going {"right" if play_going_right else "left"} starting on the {absolute_yd_line} yd line')

Play going left starting on the 89.0 yd line


# For our example we will work with the 1st play of wk 1 of the season

gameid: 2023090700
playid: 101

## Let's do some EDA

In [87]:
#first frame
first_frame = df_focused[df_focused['frame_id'] == 1]

first_frame
df_focused.columns

Index(['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id',
       'play_direction', 'absolute_yardline_number', 'player_name',
       'player_height', 'player_weight', 'player_birth_date',
       'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a',
       'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'season',
       'week', 'game_date', 'game_time_eastern', 'home_team_abbr',
       'visitor_team_abbr', 'play_description', 'quarter', 'game_clock',
       'down', 'yards_to_go', 'possession_team', 'defensive_team',
       'yardline_side', 'yardline_number', 'pre_snap_home_score',
       'pre_snap_visitor_score', 'pass_result', 'pass_length',
       'offense_formation', 'receiver_alignment', 'route_of_targeted_receiver',
       'play_action', 'dropback_type', 'dropback_distance',
       'pass_location_type', 'defenders_in_the_box', 'team_coverage_man_zone',
       'team_coverage_type'],
      dtype='object')

In [88]:
# place LOS depending on play direction and absolute_yd_line. 110 because absolute_yd_line includes endzone width

first_down_marker = (
    (line_of_scrimmage + df_focused.yards_to_go.values[0])
    if play_going_right
    else (line_of_scrimmage - df_focused.yards_to_go.values[0])
)  # Calculate 1st down marker


down = df_focused.down.values[0]
quarter = df_focused.quarter.values[0]
game_clock = df_focused.game_clock.values[0]
play_description = df_focused.play_description.values[0]


# Handle case where we have a really long Play Description and want to split it into two lines
if len(play_description.split(" ")) > 15 and len(play_description) > 115:
    play_description = (
        " ".join(play_description.split(" ")[0:16])
        + "<br>"
        + " ".join(play_description.split(" ")[16:])
    )

print(
    f"Line of Scrimmage: {line_of_scrimmage}, First Down Marker: {first_down_marker}, Down: {down}, Quarter: {quarter}, Game Clock: {game_clock}, Play Description: {play_description}"
)


Line of Scrimmage: 89.0, First Down Marker: 87.0, Down: 3, Quarter: 1, Game Clock: 12:56, Play Description: (12:56) (Shotgun) P.Mahomes pass short left to J.McKinnon to KC 31 for 10 yards (M.Rodriguez).


In [89]:
# initialize plotly play and pause buttons for animation
updatemenus_dict = [
    {
        "buttons": [
            {
                "args": [
                    None,
                    {
                        "frame": {"duration": 100, "redraw": False},
                        "fromcurrent": True,
                        "transition": {"duration": 0},
                    },
                ],
                "label": "Play",
                "method": "animate",
            },
            {
                "args": [
                    [None],
                    {
                        "frame": {"duration": 0, "redraw": False},
                        "mode": "immediate",
                        "transition": {"duration": 0},
                    },
                ],
                "label": "Pause",
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top",
    }
]

In [95]:

# initialize plotly slider to show frame position in animation
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Frame:",
        "visible": True,
        "xanchor": "right",
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": [],
}

# Frame Info
sorted_frame_list = df_focused.frame_id.unique()
sorted_frame_list.sort()

frames = []
data_history = []
for frameId in sorted_frame_list:
    data = []
    # Add Yardline Numbers to Field
    data.append(
        go.Scatter(
            x=np.arange(20, 110, 10),
            y=[5] * len(np.arange(20, 110, 10)),
            mode="text",
            text=list(
                map(str, list(np.arange(20, 61, 10) - 10) + list(np.arange(40, 9, -10)))
            ),
            textfont_size=30,
            textfont_family="Courier New, monospace",
            textfont_color="#ffffff",
            showlegend=False,
            hoverinfo="none",
        )
    )
    data.append(
        go.Scatter(
            x=np.arange(20, 110, 10),
            y=[53.5 - 5] * len(np.arange(20, 110, 10)),
            mode="text",
            text=list(
                map(str, list(np.arange(20, 61, 10) - 10) + list(np.arange(40, 9, -10)))
            ),
            textfont_size=30,
            textfont_family="Courier New, monospace",
            textfont_color="#ffffff",
            showlegend=False,
            hoverinfo="none",
        )
    )
    # Add line of scrimage
    data.append(
        go.Scatter(
            x=[line_of_scrimmage, line_of_scrimmage],
            y=[0, 53.5],
            line_dash="dash",
            line_color="blue",
            showlegend=False,
            hoverinfo="none",
        )
    )
    # Add First down line
    data.append(
        go.Scatter(
            x=[first_down_marker, first_down_marker],
            y=[0, 53.5],
            line_dash="dash",
            line_color="yellow",
            showlegend=False,
            hoverinfo="none",
        )
    )




    
    # Plot Players
    for role in df_focused.player_role.unique():

        plot_df = df_focused[
            (df_focused.player_role == role) & (df_focused.frame_id == frameId)
        ].copy()

    
        hover_text_array = []
        for nflId in plot_df.nfl_id:
            selected_player_df = plot_df[plot_df.nfl_id == nflId]
            hover_text_array.append(
                f"nflId:{selected_player_df['nfl_id'].values[0]}<br>displayName:{selected_player_df['player_name'].values[0]}"
            )
        data.append(
            go.Scatter(
                x=plot_df["x"],
                y=plot_df["y"],
                mode="markers",
                marker_color=colors[role],
                marker_size=10,
                name=role,
                hovertext=hover_text_array,
                hoverinfo="text",
            )
        )


        # Add orientation lines
        line_length = 1.25

        line_x = []
        line_y = []
        
        for i in range(len(plot_df)):
            
            # Only add line if this specific player has orientation data
            if pd.notna(plot_df["o"].iloc[i]):
                x_end_val = plot_df["x"].iloc[i] + line_length * np.sin(np.radians(plot_df["o"].iloc[i]))
                y_end_val = plot_df["y"].iloc[i] + line_length * np.cos(np.radians(plot_df["o"].iloc[i]))
                
                line_x.extend([plot_df["x"].iloc[i], x_end_val, None])
                line_y.extend([plot_df["y"].iloc[i], y_end_val, None])
        

        data.append(
            go.Scatter(
                x=line_x,
                y=line_y,
                mode="lines",
                line=dict(color=colors[role], width=2),
                showlegend=False,
                hoverinfo="skip",
            )
        )


    # plot ball landing
    data.append(
            go.Scatter(
                x=np.array(ball_land_x),
                y=np.array(ball_land_y),
                mode="markers",
                marker_color='#895129',
                marker_size=10,
                name='Ball Land',
                hovertext=['Ball!'],
                hoverinfo="text",
            )
    )

    if frameId == pass_frame_id:
        data_history.append(data)
        
    # add frame to slider
    slider_step = {
        "args": [
            [frameId],
            {
                "frame": {"duration": 100, "redraw": True},
                "mode": "immediate",
                "transition": {"duration": 0},
            },
        ],
        "label": str(frameId),
        "method": "animate",
    }
    sliders_dict["steps"].append(slider_step)
    frames.append(go.Frame(data=data, name=str(frameId)))

    
scale = 10
layout = go.Layout(
    autosize=False,
    width=120 * scale,
    height=60 * scale,
    xaxis=dict(
        range=[0, 120],
        autorange=False,
        tickmode="array",
        tickvals=np.arange(10, 111, 5).tolist(),
        showticklabels=False,
    ),
    yaxis=dict(range=[0, 53.3], autorange=False, showgrid=False, showticklabels=False),
    plot_bgcolor="#00B140",
    # Create title and add play description at the bottom of the chart for better visual appeal
    title=f"GameId: {game_id}, PlayId: {play_id}<br>{game_clock} {quarter}Q, Pass at Frame {pass_frame_id}"
    + "<br>" * 19
    + f"{play_description}",
    updatemenus=updatemenus_dict,
    sliders=[sliders_dict],
)

fig = go.Figure(data=frames[0]["data"], layout=layout, frames=frames[1:])
fig.update_layout(
    updatemenus=[{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 100, "redraw": True},
                               "fromcurrent": True, 
                               "transition": {"duration": 0}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                 "mode": "immediate",
                                 "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }]
)

# Create First Down Markers
for y_val in [0, 53]:
    fig.add_annotation(
        x=first_down_marker,
        y=y_val,
        text=str(down),
        showarrow=False,
        font=dict(family="Courier New, monospace", size=16, color="black"),
        align="center",
        bordercolor="black",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=1,
    )

In [96]:
fig.show()